In [1]:
from io import BytesIO
from PIL import Image, ImageOps
import numpy as np
import requests

from graphpipe import remote

In [2]:
import json

In [3]:
from collections import OrderedDict

In [330]:
from typing import List, Tuple, Union

In [4]:
def lord_labels():
    """
    """
    filename = 'imagenet_class_index.json'
    with open(filename, 'r') as fr:
        labels = json.load(fr, object_pairs_hook=OrderedDict)
    return [v[-1] for k, v in labels.items()]

In [5]:
labels = lord_labels()

In [6]:
def resize(img, size):
    """
    保持长宽比缩放到 size, 周围可以用 (0, 0, 0) 填充
    :param img:
    :param size:
    :return:
    """
    if isinstance(size, int):
        size = (size, size)
    w, h = img.size
    scale = min(size[0]/w, size[1]/h)
    width = int(w * scale)
    height = int(h * scale)
    img = img.resize((width, height), Image.LANCZOS)

    _img = Image.new("RGB", size)
    _img.paste(img, ((size[0] - width) // 2, (size[1] - height) // 2))
    return _img

In [7]:
def preprocess(filename, input_size):
    """
    """
    img = Image.open(filename)
    img = img.convert('RGB')
    img = resize(img, input_size)
    data = np.array(img)
    data = np.expand_dims(data, axis=0)
    data = np.einsum('nwhc -> ncwh', data)
    return data.astype(dtype=np.float32)

In [77]:
img_filename = 'C:/Users/emile/Pictures/mug227.png'
# img_filename = 'C:/Users/emile/Pictures/test_0.png'

In [117]:
def prediction(filename):
    """
    """
    data = preprocess(filename, 227)
    pred = remote.execute("http://192.168.10.6:9000", data)
    pred = pred[0]
    top_5_ind = np.argpartition(pred, -5)[-5:]
    top_5 = top_5_ind[np.argsort(pred[top_5_ind])][::-1]
    for i in top_5:
        label = labels[i]
        prob = pred[i]
        print(f'label: {label}, prob: {prob}')

In [118]:
prediction(img_filename)

label: coffee_mug, prob: 0.88107830286026
label: cup, prob: 0.1086273193359375
label: coffeepot, prob: 0.002403728896752
label: whiskey_jug, prob: 0.0017930584726855159
label: water_jug, prob: 0.0017816754989326


In [80]:
import torch
import torchvision

In [81]:
from torch import nn

In [82]:
net = torchvision.models.densenet121(pretrained=True)
net.eval()

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplac

In [83]:
transform = torchvision.transforms.Compose(
    [torchvision.transforms.ToTensor(),
     torchvision.transforms.Normalize((0.49140089750289917, 0.4821591377258301, 0.4465310275554657),
                                      (0.24702748656272888, 0.24348321557044983, 0.26158758997917175))
    ])

In [84]:
def normalize(filename, input_size):
    img = Image.open(filename)
    img = img.convert('RGB')
    img = resize(img, input_size)
    data = transform(img)
    return data.unsqueeze(dim=0)

In [90]:
data = normalize(img_filename, 227)

In [91]:
pred = net(data)

In [92]:
pred = nn.Softmax(dim=1)(pred)[0]

In [88]:
pred = pred.topk(k=5)

In [89]:
for i in range(5):
    index = pred.indices[i].item()
    prob = pred.values[i]
    label = labels[index]
    print(f'label: {label}, prob: {prob}')

label: coffee_mug, prob: 0.5072744488716125
label: espresso, prob: 0.29593625664711
label: cup, prob: 0.055702388286590576
label: whiskey_jug, prob: 0.046067655086517334
label: tray, prob: 0.008652608841657639


In [331]:
def topk_along_axis(a: np.ndarray,
                    kth: int,
                    axis: int = -1,
                    kind: str = 'introselect',
                    order: Union[str, List[str]] = None,
                    largest: bool = True,
                    sort: bool = True) -> Tuple[np.ndarray, np.ndarray]:
    """
    沿坐标轴 axis 取前 kth 个值, 默认 (largest 为 True) 取最大的 kth 个
    :param a:
    :param kth:
    :param axis:
    :param kind:
    :param order:
    :param largest: True: 最大的 kth 个, False: 最小的 kth 个
    :param sort: 输出是否排序
    :return: 输出 idx, values
    """
    shape = a.shape
    if not largest:
        idx = np.argpartition(a, kth, axis=axis, kind=kind, order=order).take(range(0, kth), axis=axis)
    else:
        idx = np.argpartition(a, -kth, axis=axis, kind=kind, order=order).take(range(shape[axis] - kth, shape[axis]),
                                                                               axis=axis)

    if sort:
        idx = np.take_along_axis(idx,
                                 np.argsort(np.take_along_axis(a, idx, axis=axis), axis=axis),
                                 axis=axis)
        if largest:
            idx = np.flip(idx, axis=axis)
    return idx, np.take_along_axis(a, idx, axis=axis)

In [345]:
def topk(a: np.ndarray,
         kth: int,
         kind: str = 'introselect',
         order: Union[str, List[str]] = None,
         largest: bool = True,
         sort: bool = True) -> Tuple[np.ndarray, np.ndarray]:
    """
    在 a 的所有项中取前 kth 项, 默认 (largest 为 True) 取最大的 kth 个
    :param a:
    :param kth:
    :param kind:
    :param order:
    :param largest: True: 最大的 kth 个, False: 最小的 kth 个
    :param sort: 输出是否排序
    :return: 输出 idx, values
    """
    base_a = a.ravel()
    if largest:
        idx = np.argpartition(base_a, a.size - kth, kind=kind, order=order)[-kth:]
    else:
        idx = np.argpartition(base_a, kth, kind=kind, order=order)[:kth]

    if sort:
        idx = idx[np.argsort(base_a[idx])]
        if largest:
            idx = idx[::-1]
    retrieval_idx = np.unravel_index(idx, a.shape)
    idx = np.column_stack(retrieval_idx)
    return idx, a[retrieval_idx]

In [346]:
x = np.random.normal(size=(2,10))

In [347]:
x

array([[ 0.29054321, -1.02376525, -0.09419705,  1.26074149,  0.29756626,
        -0.47101124,  2.2747721 , -0.97398114, -0.53942564, -0.89383041],
       [ 0.59671197, -0.51507811,  1.52055004, -0.70413789, -0.92309623,
        -0.3880049 , -0.56809266, -1.5252515 , -1.15024188,  1.32951315]])

In [348]:
topk_along_axis(x, 3, axis=-1)

(array([[6, 3, 4],
        [2, 9, 0]], dtype=int64), array([[2.2747721 , 1.26074149, 0.29756626],
        [1.52055004, 1.32951315, 0.59671197]]))

In [349]:
topk(x, 3)

(array([[0, 6],
        [1, 2],
        [1, 9]], dtype=int64), array([2.2747721 , 1.52055004, 1.32951315]))

In [350]:
x[0]

array([ 0.29054321, -1.02376525, -0.09419705,  1.26074149,  0.29756626,
       -0.47101124,  2.2747721 , -0.97398114, -0.53942564, -0.89383041])

In [308]:
top_index = np.argpartition(x, -3).take(indices=range(l-3, l), axis=-1)

In [309]:
top_index

array([[1, 8, 5],
       [3, 1, 4]], dtype=int64)

In [323]:
np.flip(top_index, axis=0)

array([[3, 1, 4],
       [1, 8, 5]], dtype=int64)

In [317]:
idx = np.take_along_axis(top_index, np.argsort(np.take_along_axis(x, top_index, axis=-1), axis=-1), axis=-1)

In [328]:
idx

array([[1, 8, 5],
       [3, 1, 4]], dtype=int64)

In [318]:
x[np.array([[0], [1]]), idx]

array([[0.67770244, 0.86220759, 0.94449671],
       [0.77546003, 1.10925011, 1.44256386]])

In [211]:
idx = np.array([[0, 0, 0],
               [1, 1, 1]])

In [210]:
x[1, 6]

1.122107472880876

In [212]:
top_index

array([[8, 3, 1],
       [6, 8, 7]], dtype=int64)

In [243]:
np.indices((2, 10))

array([[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]],

       [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]]])

In [239]:
x[idx, top_index]

array([[0.71346409, 0.86443531, 1.00370275],
       [1.12210747, 1.25317466, 2.12692661]])

In [240]:
x[..., top_index]

array([[[ 0.71346409,  0.86443531,  1.00370275],
        [ 0.04821987,  0.71346409, -0.46542615]],

       [[ 1.25317466, -1.48457252, -0.25440349],
        [ 1.12210747,  1.25317466,  2.12692661]]])

In [185]:
np.take_along_axis(x, top_index, axis=-1)

array([[-1.53987514, -1.47566612, -0.46542615, -0.45516066, -0.30825544,
         0.04821987,  0.40089877,  0.71346409,  0.86443531,  1.00370275],
       [-1.48457252, -1.2320771 , -0.77019231, -0.25440349,  0.0283222 ,
         0.24003861,  0.75777865,  1.12210747,  1.25317466,  2.12692661]])

In [170]:
top_index

array([2, 8, 6], dtype=int64)

In [237]:
topk(x, 3)

IndexError: index 9 is out of bounds for axis 0 with size 2

In [235]:
arr = np.array([[0, 1, 2],
                [0, 0, 1]])

In [236]:
np.ravel_multi_index(arr, (8, 6))

array([ 0,  6, 13], dtype=int64)

In [215]:
np.ravel_multi_index(x, 2)

ValueError: parameter multi_index must be a sequence of length 1

In [265]:
 y = np.arange(35).reshape(5,7)

In [266]:
y

array([[ 0,  1,  2,  3,  4,  5,  6],
       [ 7,  8,  9, 10, 11, 12, 13],
       [14, 15, 16, 17, 18, 19, 20],
       [21, 22, 23, 24, 25, 26, 27],
       [28, 29, 30, 31, 32, 33, 34]])

In [276]:
y.take(np.array([2, 3, 4]), axis=-1)

array([[ 2,  3,  4],
       [ 9, 10, 11],
       [16, 17, 18],
       [23, 24, 25],
       [30, 31, 32]])

In [287]:
np.arange(y.shape[0]).reshape((y.shape[0], 1))

array([[0],
       [1],
       [2],
       [3],
       [4]])